In [32]:
import grequests
import os
import json
import pandas as pd

API_ENDPOINT="https://api.osmosis.interbloc.org/cosmos/tx/v1beta1/txs?events=tx.height={}"

UPGRADE_HEIGHT=4707300
HALT_HEIGHT=4713064

MSG_EXIT_POOL="/osmosis.gamm.v1beta1.MsgExitPool"
MSG_JOIN_POOL="/osmosis.gamm.v1beta1.MsgJoinPool"

### Download all the data 

Download all tx responses for every block in [`UPGRADE_HEIGHT`, `HALT_HEIGHT`] to `data/block_tx/`

In [33]:
FILE_PATH = 'data/block_tx/{}.json'
STEP = 5

for height in range(UPGRADE_HEIGHT, HALT_HEIGHT, STEP):

    urls = [API_ENDPOINT.format(h) for h in range(height, height + STEP) if not os.path.exists(FILE_PATH.format(h)) and h <= HALT_HEIGHT]
    rs = (grequests.get(url, headers={'Accept': 'application/json'}) for url in urls)
    responses = grequests.map(rs, size = STEP)

    for idx, response in enumerate(responses):

        curr_height = response.request.url.replace("https://api.osmosis.interbloc.org/cosmos/tx/v1beta1/txs?events=tx.height=", "")
        with open(FILE_PATH.format(curr_height), 'w') as f:
            json.dump(response.json()["tx_responses"], f)


## Post Processing data

In [ ]:

# for tx in response_json["tx_responses"]:
#     results.append(tx)

# raw_df = pd.DataFrame.from_records(results)
# raw_df.to_csv("data/raw_txs.csv")

### Process the data

In [ ]:
# Take only the needed columns
df = raw_df[["height","txhash", "code", "timestamp","tx"]]

# Remove unsuccessful transactions
df = df[df["code"] == 0]


In [ ]:
# Expand tx
df = df.join(pd.json_normalize(df.tx)[["body.messages"]]).drop(columns=["tx"])
df = df.explode(column="body.messages")
df.head()


In [ ]:
df = df.join(pd.json_normalize(df["body.messages"])[["@type", "poolId","shareInAmount","shareOutAmount", "sender", "tokenInMaxs", "tokenOutMins"]])
df = df.drop(columns=["body.messages"])

# Filter messages
df = df[(df["@type"] == MSG_EXIT_POOL) | (df["@type"] == MSG_JOIN_POOL )]
df.head()


In [ ]:
# Merge shares into one column
df["shares"] = df["shareInAmount"].combine_first(df["shareOutAmount"])
df.drop(columns=["shareInAmount", "shareOutAmount"], inplace=True)

# Merge tokens into one column
df["tokens"] = df["tokenInMaxs"].combine_first(df["tokenOutMins"])
df.drop(columns=["tokenInMaxs","tokenOutMins"], inplace=True)

In [ ]:
# Expand token information
df.reset_index(inplace=True, drop=True)
df = df.join(pd.json_normalize(pd.json_normalize(df["tokens"])[0]).add_prefix("token0_"))
df = df.join(pd.json_normalize(pd.json_normalize(df["tokens"])[1]).add_prefix("token1_"))
df.drop(columns=["tokens"], inplace=True)
display(df)

## Calculate Excess GAMM

## Count of Join/Exit Transactions

In [ ]:
df.groupby('@type').size().reset_index(name='number_of_txs')

In [ ]:
(df.groupby(['@type', 'sender'])
    .size()
    .reset_index(name='number_of_txs')
    .sort_values(by=['number_of_txs'], ascending=False))